# 1. Label Training Image

In [101]:
!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: /Users/sarvzz/.labelmerc


# 2. Apply Image Augmentation on Images and Labels using Albumentations

### 2.1 Setup Alumentation Pipeline

In [1]:
import albumentations as alb

In [2]:
# checking our original image size
import cv2
import os

# img = cv2.imread(os.path.join('V16-2023-10-12','data','images','frame_42.jpg'))
# img_size = img.shape
# img_size

In [3]:
augmentor = alb.Compose([#alb.CenterCrop(width=700, height=700), 
                         #alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2)], 
                         bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

### 2.2 Load a Test Image

In [71]:
img = cv2.imread(os.path.join('V16-2023-10-12','data','images','frame_00.jpg'))

In [72]:
import json

with open(os.path.join('V16-2023-10-12','data','labels','frame_00.json'),'r') as f:
    label = json.load(f)

In [73]:
x1_normalized = (label['shapes'][0]['points'][0][0])/700    #LeftEye   (x1)
y1_normalized = (label['shapes'][0]['points'][0][1])/700    #LeftEye   (y1)
x2_normalized = (label['shapes'][0]['points'][1][0])/700    #LeftEye   (x2)
y2_normalized = (label['shapes'][0]['points'][1][1])/700    #LeftEye   (y2)

x3_normalized = (label['shapes'][1]['points'][0][0])/700    #RightEye  (x3)
y3_normalized = (label['shapes'][1]['points'][0][1])/700    #RightEye  (y3)
x4_normalized = (label['shapes'][1]['points'][1][0])/700    #RightEye  (x4)
y4_normalized = (label['shapes'][1]['points'][1][1])/700    #RightEye  (y4)


# Create a list of normalized bounding box coordinates
# LeftEye_coords = [x1_normalized, y1_normalized, x2_normalized, y2_normalized]
# RightEye_coords = [x3_normalized, y3_normalized, x4_normalized, y4_normalized]

# Create a list of bboxes with LeftEye and RightEye coordinates
bboxes = [(x1_normalized, y1_normalized, x2_normalized, y2_normalized),(x3_normalized, y3_normalized, x4_normalized, y4_normalized)]
bboxes

[(0.492358803986711,
  0.3770321151716501,
  0.5444075304540421,
  0.42741971207087487),
 (0.24928017718715392,
  0.37758582502768545,
  0.3002214839424142,
  0.426312292358804)]

In [74]:
augmented = augmentor(image=img, bboxes=bboxes, class_labels=['Iris','Iris'])
augmented

{'image': array([[[120, 153, 175],
         [120, 153, 175],
         [120, 153, 175],
         ...,
         [103, 132, 149],
         [103, 132, 149],
         [103, 132, 149]],
 
        [[120, 153, 175],
         [120, 153, 175],
         [120, 153, 175],
         ...,
         [103, 132, 149],
         [103, 132, 149],
         [103, 132, 149]],
 
        [[120, 153, 175],
         [120, 153, 175],
         [120, 153, 175],
         ...,
         [103, 132, 149],
         [103, 132, 149],
         [103, 132, 149]],
 
        ...,
 
        [[138, 130, 173],
         [164, 159, 201],
         [178, 175, 217],
         ...,
         [148, 121, 153],
         [144, 119, 151],
         [134, 110, 139]],
 
        [[135, 128, 173],
         [165, 161, 206],
         [174, 172, 217],
         ...,
         [149, 122, 154],
         [146, 121, 153],
         [131, 108, 137]],
 
        [[137, 133, 177],
         [167, 165, 211],
         [171, 170, 217],
         ...,
         [143, 118,

In [77]:
import numpy as np
from matplotlib import pyplot as plt

cv2.rectangle(augmented['image'], 
              tuple((augmented['bboxes'][0][:2])),
              tuple((augmented['bboxes'][0][2:])),
              (0, 255, 0), 2)

              # tuple(np.multiply(augmented['bboxes'][0][:2], [450, 450]).astype(int)),
              # tuple(np.multiply(augmented['bboxes'][0][2:], [450, 450]).astype(int)),
              # (0, 255, 0), 2)

cv2.rectangle(augmented['image'], 
              tuple((augmented['bboxes'][1][:2])),
              tuple((augmented['bboxes'][1][2:])),
              (0, 255, 0), 2)

              # tuple(np.multiply(augmented['bboxes'][1][:2], [450, 450]).astype(int)),
              # tuple(np.multiply(augmented['bboxes'][1][2:], [450, 450]).astype(int)),
              # (0, 255, 0), 2)


plt.imshow(augmented['image'])

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


### 2.3 Run Augmentation Pipeline

In [96]:
import cv2
import os
import json
import numpy as np
from albumentations import Compose, BboxParams

#for partition in ['train', 'test', 'validate']:
for partition in ['train']:
    image_folder = os.path.join('data-1', partition, 'images')
    label_folder = os.path.join('data-1', partition, 'labels')

    for image in os.listdir(image_folder):
        img_path = os.path.join(image_folder, image)
        img = cv2.imread(img_path)


        #bboxes = [[0, 0, 0.00001, 0.00001], [0, 0, 0.00001, 0.00001]] 
        label_path = os.path.join(label_folder, f'{os.path.splitext(image)[0]}.json')

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            if len(label['shapes']) >= 2:

                x1_normalized = (label['shapes'][0]['points'][0][0]) #/450    #LeftEye   (x1)
                y1_normalized = (label['shapes'][0]['points'][0][1]) #/450    #LeftEye   (y1)
                x2_normalized = (label['shapes'][0]['points'][1][0]) #/450    #LeftEye   (x2)
                y2_normalized = (label['shapes'][0]['points'][1][1]) #/450    #LeftEye   (y2)
                
                x3_normalized = (label['shapes'][1]['points'][0][0]) #/450    #RightEye  (x3)
                y3_normalized = (label['shapes'][1]['points'][0][1]) #/450    #RightEye  (y3)
                x4_normalized = (label['shapes'][1]['points'][1][0]) #/450    #RightEye  (x4)
                y4_normalized = (label['shapes'][1]['points'][1][1]) #/450    #RightEye  (y4)

            else: 
                x1_normalized = 0  
                y1_normalized = 0
                x2_normalized = 0
                y2_normalized = 0
                x3_normalized = 0 
                y3_normalized = 0
                x4_normalized = 0
                y4_normalized = 0

            
            # Create a list of bboxes with LeftEye and RightEye coordinates
            bboxes = [(x1_normalized, y1_normalized, x2_normalized, y2_normalized),(x3_normalized, y3_normalized, x4_normalized, y4_normalized)]
        

        try: 
            for x in range(2):
                augmented = augmentor(image=img, bboxes=bboxes, class_labels=['Iris', 'Iris'])
                cv2.imwrite(os.path.join('V16-2023-10-12','data','aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [[0,0,0,0],[0,0,0,0]]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = [augmented['bboxes'][0],augmented['bboxes'][1]]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [[0,0,0,0],[0,0,0,0]]
                    annotation['class'] = 0 


                with open(os.path.join('V16-2023-10-12','data','aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

image must be numpy array type


# 3. LabelMe to COCO

### 3.1 Convert LabelMe to COCO

In [3]:
pip install -U labelme2coco

  Using cached labelme2coco-0.2.4-py3-none-any.whl (19 kB)
  Obtaining dependency information for sahi>=0.8.19 from https://files.pythonhosted.org/packages/bf/2b/026f0861137499edca43263679f9e4ce21314b67c34c8d74c12f2833b8c3/sahi-0.11.14-py3-none-any.whl.metadata
  Using cached sahi-0.11.14-py3-none-any.whl.metadata (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.3 MB/s eta 0:00:0000:0100:01
  Using cached pybboxes-0.1.6-py3-none-any.whl (24 kB)
  Using cached fire-0.5.0.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
Using cached sahi-0.11.14-py3-none-any.whl (104 kB)
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116931 sha256=bbd96e4a786b9c24b67b89c0ab84da5ae6a5e0e4e29471d15b53375984b289f8
  Stored in directory: /Users/sarvzz/Library/Caches/pip/wheels/a7/ee/a5/19e91481be8bea594935d137578bfe77d6bf905e4595336f6b
Successfully built fire
Note: you may need to restart the 

In [24]:
import labelme2coco 
import os
import json
#from pycocotools.coco import COCO
#from pycocotools import mask as maskUtils

In [78]:
# input folder 
labelme_folder = 'V16-2023-10-12/data/labels'

# output folder
#export_dir = 'V10-2023-09-27/data/train/coco_annotations/coco_annotation.json'
export_dir = 'V16-2023-10-12/data/coco_annotations'

# convert labelme annotations to coco
labelme2coco.convert(labelme_folder, export_dir)

There are 450 listed files in folder labels.


Converting labelme annotations to COCO format: 100%|█| 450/450 [00:00<00:00, 166
10/13/2023 10:20:12 - INFO - labelme2coco -   Converted annotations in COCO format is exported to V16-2023-10-12/data/coco_annotations/dataset.json


# 4. Change 'imagePath' in .json file to it's file name

In [2]:
import json
import os

# Labelme directory
json_dir = 'upload_data/labels/'

# Iterate through all json files
for json_file in os.listdir(json_dir):
    if json_file.endswith('.json'):
        
        # Read the json file 
        with open(os.path.join(json_dir, json_file), 'r') as f:
            labelme_data = json.load(f)

        
        # Extract json file name
        json_file_name = os.path.splitext(json_file)[0]
        
        # Update the 'imagePath' with .jpg
        labelme_data['imagePath'] = f'{json_file_name}.jpg'

        # Write the updated json data back to the file
        with open(os.path.join(json_dir, json_file), 'w') as f:
            json.dump(labelme_data, f, indent=4)
    
    # else:
    #     print(f'skipping empty file: {json_file_path}')

# 5. VOC XML to COCO

In [ ]:
import json 
import xml.etree.ElementTree as ET

# Define path to PASCAL VOC XML file
xml_file = 'annotated images/xml annotations/annotations.xml'

# Initialize the COCO-format dictionary
coco_data = {
    'images': [],
    'annotations': [],
    'categories': []
}

# Mapping of class names to COCO category IDs
class_to_id = {
    'Pupilmargin': 1,
    'PupilCenter': 2
}

# Parse the XML file
tree = ET.parse(xml_file)
root = tree.getroot()

# Iterate through the 'image' elements in the XML
for image_elem in root.findall('image'):
    image_info = {
        'id': int(image_elem.get('id')),
        'file_name': image_elem.get('name'),
        'width': int(image_elem.get('width')),
        'height': int(image_elem.get('height'))
        
    }

coco_data['images'].append(image_info)

# Extract object annotations for each image
for box_elem in image_elem.findall('box'):
    category_name = box_elem.get('label')
    

# 6. JSON File Name - Update

In [97]:
import os
import shutil 

In [98]:
# Function to create copies and update file extension 
def duplicate_files_with_extensions(file_path, num_copies):
    # Split the file path into base name and extension
    base_name, file_extension = file_path.rsplit('.', 1)

    # Create copies of the file with different extensions
    for i in range(num_copies):
        new_file_name = f'{base_name}.{i}.{file_extension}'
        shutil.copy(file_path, new_file_name)

In [100]:
# file updates
file_path = 'V16-2023-10-12/data/testing_files/frame_00.json'
num_copies = 3

duplicate_files_with_extensions(file_path, num_copies)